# 使用在线终结点进行(视觉)聊天补全推理

本示例展示了如何将 `Phi-3-vision-128k-instruct` 部署到在线总结点进行推理。

## 大纲
* 设置先决条件
* 选择要部署的模型
* 下载并准备推理数据
* 部署模型以实现实时推理
* 测试终结点
* 使用 Azure OpenAI 风格的负载测试终结点
* 清理资源

## 1. 设置先决条件
* 安装依赖项
* 连接到 AzureML 工作区。了解更多信息请参阅[设置 SDK 认证](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk)。替换下文代码中的 `<WORKSPACE_NAME>`、`<RESOURCE_GROUP>` 和 `<SUBSCRIPTION_ID>`。
* 连接到 `azureml` 系统注册表

In [ ]:
# I导入所需的库
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    # 尝试获得默认凭据
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # 如果默认凭据不可用，尝试使用交互式浏览器凭据
    credential = InteractiveBrowserCredential()

try:
    # 尝试使用凭据创建MLClient实例
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # 如果创建MLClient实例失败，使用手动提供的信息
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<WORKSPACE_NAME>"

# 用之前提供的凭据和工作区信息创建MLClient实例
workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# 模型、微调管道和环境可在 AzureML 系统注册表 "azureml" 中找到
registry_ml_client = MLClient(credential, registry_name="azureml")

## 2. 部署模型到在线终结点

在线端点提供一个持久的REST API，可以用于与需要使用该模型的应用程序进行集成。

In [ ]:
# 这段代码检查注册表中是否存在指定名称的模型。
# 如果模型存在，它会检索模型的第一个版本并打印其详细信息。
# 如果模型不存在，它会打印一条消息，表明该模型未找到。

# model_name: 要检查的注册表中的模型名称
model_name = "Phi-3-vision-128k-instruct"

# 获取指定型号名称的版本列表
version_list = list(registry_ml_client.models.list(model_name))

# 检查注册表中是否存在模型的任何版本
if len(version_list) == 0:
    print("Model not found in registry")
else:
    # 获取模型的第一个版本
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    
    # 打印模型的详细信息
    print(
        "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
            foundation_model.name, foundation_model.version, foundation_model.id
        )
    )

In [ ]:
# 导入所需的库
import time
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# 端点名称在一个区域内必须是唯一的，因此使用时间戳来创建唯一的端点名称
timestamp = int(time.time())
online_endpoint_name = model_name[:13] + str(timestamp)
print(f"Creating online endpoint with name: {online_endpoint_name}")

# 创建在线终结点
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description=f"Online endpoint for {foundation_model.name}, for visual chat-completion task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# 此代码将部署在线终结点。
# 它将设置部署名称、端点名称、模型、实例类型、实例数量和请求设置。
# 还会设置有效性探针和就绪性探针。
# 最后，更新端点的流量分布。

from azure.ai.ml.entities import OnlineRequestSettings, ProbeSettings

# create a deployment
deployment_name = "phi-3-vision"
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_NC48ads_A100_v4",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=180000,
        max_queue_wait_ms=500,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=49,
        success_threshold=1,
        timeout=299,
        period=180,
        initial_delay=180,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=10,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

## 3. 使用样本数据测试终结点

我们将使用下面创建的 json 向模型发送一个示例请求。

In [ ]:
# 导入所需的库
import json
import os

# 定义测试 JSON
test_json = {
    "input_data": {
        "input_string": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                        },
                    },
                    {
                        "type": "text",
                        "text": "What is shown in this image? Be extremely detailed and specific.",
                    },
                ],
            },
        ],
        "parameters": {"temperature": 0.7, "max_new_tokens": 2048},
    }
}

# 将 JSON 对象写入文件
sample_score_file_path = os.path.join(".", "sample_chat_completions_score.json")
with open(sample_score_file_path, "w") as f:
    json.dump(test_json, f, indent=4)

# 打印输入负载
print("Input payload:\n")
print(test_json)

In [ ]:
# 导入所需的库
import pandas as pd

# 使用在线终结点和 azureml 终结点的 invoke 方法为 sample_chat_completions_score.json 文件评分
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file=sample_score_file_path,
)
print("Raw JSON Response: \n", response, "\n")

# Parse the JSON string
json_data = json.loads(response)

# Convert the parsed JSON to a DataFrame
response_df = pd.DataFrame([json_data])
print("Generated Text:\n", response_df["output"].iloc[0])

## 4. 用 Azure OpenAI 风格的负载测试终结点

我们将向模型发送一条 Azure OpenAI 风格的示例请求。

In [ ]:
# 此代码定义了一个 JSON 载荷，用于使用 Azure OpenAI 风格的载荷测试在线端点。
# 它包括模型名称、带有用户角色和内容（图片 URL 和文本）的消息列表、
# 温度和最大生成令牌数。

aoai_test_json = {
    "model": foundation_model.name,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                    },
                },
                {
                    "type": "text",
                    "text": "What is shown in this image? Be extremely detailed and specific.",
                },
            ],
        }
    ],
    "temperature": 0.7,
    "max_new_tokens": 2048,
}

In [ ]:
# 获取得分 URL
scoring_uri = workspace_ml_client.online_endpoints.get(
    name=online_endpoint_name
).scoring_uri
# 将得分 URL 转换为 AOAI 格式
aoai_format_scoring_uri = scoring_uri.replace("/score", "/v1/chat/completions")

# 获取数据平面操作的密钥
data_plane_token = workspace_ml_client.online_endpoints.get_keys(
    name=online_endpoint_name
).primary_key

In [ ]:
import urllib.request
import json

# 准备请求
body = str.encode(json.dumps(aoai_test_json))
url = aoai_format_scoring_uri
api_key = data_plane_token

headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + api_key)}
req = urllib.request.Request(url, body, headers)

# 发生请求并获得响应
try:
    response = urllib.request.urlopen(req)
    result = response.read().decode("utf-8")
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # 打印标头--其中包括请求 ID 和时间戳，用于调试
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

## 5. 删除在线终结点

不要忘记删除在线终结点，否则该端点会一直产生费用。

In [ ]:
# 删除工作空间
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait(#)